# Chafer Activity
Detects Chafer activity attributed to OilRig as reported in Nyotron report in March 2018

## Rule Content
```
- action: global
  title: Chafer Activity
  id: 53ba33fd-3a50-4468-a5ef-c583635cfa92
  description: Detects Chafer activity attributed to OilRig as reported in Nyotron
    report in March 2018
  references:
  - https://nyotron.com/nyotron-discovers-next-generation-oilrig-attacks/
  tags:
  - attack.persistence
  - attack.g0049
  - attack.t1053
  - attack.s0111
  - attack.defense_evasion
  - attack.t1112
  date: 2018/03/23
  modified: 2019/03/01
  author: Florian Roth, Markus Neis
  detection:
    condition: 1 of them
  falsepositives:
  - Unknown
  level: critical
- logsource:
    product: windows
    service: system
  detection:
    selection_service:
      EventID: 7045
      ServiceName:
      - SC Scheduled Scan
      - UpdatMachine
- logsource:
    product: windows
    service: security
  detection:
    selection_service:
      EventID: 4698
      TaskName:
      - SC Scheduled Scan
      - UpdatMachine
- logsource:
    product: windows
    service: sysmon
  detection:
    selection_reg1:
      EventID: 13
      TargetObject:
      - '*SOFTWARE\Microsoft\Windows\CurrentVersion\UMe'
      - '*SOFTWARE\Microsoft\Windows\CurrentVersion\UT'
      EventType: SetValue
    selection_reg2:
      EventID: 13
      TargetObject: '*\Control\SecurityProviders\WDigest\UseLogonCredential'
      EventType: SetValue
      Details: DWORD (0x00000001)
- logsource:
    category: process_creation
    product: windows
  detection:
    selection_process1:
      CommandLine:
      - '*\Service.exe i'
      - '*\Service.exe u'
      - '*\microsoft\Taskbar\autoit3.exe'
      - C:\wsc.exe*
    selection_process2:
      Image: '*\Windows\Temp\DB\\*.exe'
    selection_process3:
      CommandLine: '*\nslookup.exe -q=TXT*'
      ParentImage: '*\Autoit*'

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"7045" AND service_name:("SC\ Scheduled\ Scan" OR "UpdatMachine"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4698" AND task_name:("SC\ Scheduled\ Scan" OR "UpdatMachine"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND event_type:"SetValue" AND (registry_key_path.keyword:(*SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\UMe OR *SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\UT) OR (registry_key_path.keyword:*\\Control\\SecurityProviders\\WDigest\\UseLogonCredential AND registry_key_value:"DWORD\ \(0x00000001\)")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*\\Service.exe\ i OR *\\Service.exe\ u OR *\\microsoft\\Taskbar\\autoit3.exe OR C\:\\wsc.exe*) OR process_path.keyword:*\\Windows\\Temp\\DB\\*.exe OR (process_command_line.keyword:*\\nslookup.exe\ \-q\=TXT* AND process_parent_path.keyword:*\\Autoit*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()